In [1]:
import nltk
import os
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
wordnet_lemmatizer = WordNetLemmatizer()
package_dir = os.getcwd()

[nltk_data] Downloading package punkt to /home/madhav/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/madhav/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
input_file = os.path.join(package_dir,'Input.pdf')
fp1 = open(input_file, 'rb')
from pdfminer.pdfparser import PDFParser, PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams, LTTextBox, LTTextLine
extracted_text = ''
parser = PDFParser(fp1)
doc = PDFDocument()
parser.set_document(doc)
doc.set_parser(parser)
doc.initialize('')
rsrcmgr = PDFResourceManager()
laparams = LAParams()
laparams.char_margin = 1.0
laparams.word_margin = 1.0
device = PDFPageAggregator(rsrcmgr, laparams=laparams)
interpreter = PDFPageInterpreter(rsrcmgr, device)

for page in doc.get_pages():
    interpreter.process_page(page)
    layout = device.get_result()
    for lt_obj in layout:
        if isinstance(lt_obj, LTTextBox) or isinstance(lt_obj, LTTextLine):
            extracted_text += lt_obj.get_text()
text1 = extracted_text

In [3]:

liabilities = ['cost', 'loss', 'expense', 'debt', 'liability', 'risk', 'loan', 'outflow', 'NOL', 'net operating loss', 'divest', 'net loss', 'borrowings', 'price', 'charge',
'cost', 'expenditure', 'outlay', 'figure', 'impost', 'dues', 'tariff', 'valuation', 'appraisement', 'quotation', 'fare', 'hire', 'wages', 
'return', 'disbursement', 'rate', 'appraisal', 'reckoning', 'equivalent', 'payment', 'demand', 'barter', 'consideration', 'amount', 'credit', 'advance'
'mortgage', 'overdraft', 'debenture', 'lending', 'advance']

misc1 = ['market','share','quarter','fiscal','RMS','rate','average','estimate','tax','audit','interest','principle','budget','bill','finance','annual','business',
'franchisee','cash','ROI','UDF','contribution','dividend','division','interest','percentage','proportion','stake','budgetary','economic','banking','commerce',
'supply','demand','value','worth','deal']

assets = [
'revenue', 'profit', 'sale', 'credit', 'balance', 'asset', 'investment', 'amount', 'inflow', 'fund', 'sell', 'average final compensation', 'AFC', 'EBITDA', 
'acquisition', 'cost of goods sold', 'COGS', 'EVA', 'Economic value added', 'NP', 'net profit', 'operating income', 'income', 'return', 'yield', 'wealth', 'receipt', 
'stock', 'credit', 'dividend', 'perquisite', 'salary', 'annuity', 'acquirement', 'rent', 'gross revenue', 'appraisal', 'market price', 'grant', 'comepensation'
]

In [4]:
f2 = open('Preprocess.txt','w')
ne_tree = nltk.ne_chunk(pos_tag(word_tokenize(text1)))
for tree_ob in ne_tree:
    if(isinstance(tree_ob,tuple)):
        word1 = wordnet_lemmatizer.lemmatize(tree_ob[0])
        if word1 in liabilities:
            print(tree_ob[0]+' '+tree_ob[1]+" FLIA",file = f2)
            print(tree_ob[0]+' '+tree_ob[1]+" FLIA")
        elif word1 in assets:
            print(tree_ob[0]+' '+tree_ob[1]+" FASS",file = f2)
            print(tree_ob[0]+' '+tree_ob[1]+" FASS")
        elif word1 in misc1:
            print(tree_ob[0]+' '+tree_ob[1]+" FMIS",file = f2)
            print(tree_ob[0]+' '+tree_ob[1]+" FMIS")
        else:
            print(tree_ob[0]+' '+tree_ob[1]+" O",file = f2)
    else:
        for tree in tree_ob:
            print(tree[0]+' '+tree[1]+' '+tree_ob.label(),file = f2)
f2.close()

sells VBZ FASS
price NN FLIA
average JJ FMIS
profit NN FASS
market NN FMIS
share NN FMIS


In [5]:
import sklearn_crfsuite
test_data = open('Preprocess.txt','r')
test_sents = list(test_data)


def word2features(sent):
    word = sent[0]
    postag = sent[1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    return features


def sent2features(sent):
    sent1 = sent.split()
    return [word2features(sent1)]

def sent2labels(sent):
    sent1 = sent.split()
    return [sent1[2]]

def sent2tokens(sent):
    sent1 = sent.split()
    return [sent1[0]]


In [6]:
X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

In [7]:
import joblib
crf = joblib.load('NER.joblib')
labels = list(crf.classes_)

In [8]:
y_pred = crf.predict(X_test)
print(metrics.flat_f1_score(y_test, y_pred,
                      average='weighted', labels=labels))

fin_ner_words = []
for i in range(0,len(y_pred)):
    tup = (X_test[i][0]['word.lower()'],str(y_pred[i][0]))
    fin_ner_words.append(tup)
    
#print(fin_ner_words)


0.8739202999205768


/home/madhav/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/madhav/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [9]:
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize
nermodel = os.path.join(package_dir,'stanford-ner-2018-10-16/classifiers/english.muc.7class.distsim.crf.ser.gz')
nerjar = os.path.join(package_dir,'stanford-ner-2018-10-16/stanford-ner.jar')
st = StanfordNERTagger(nermodel,nerjar,encoding='utf-8')

text = extracted_text
text1 = ''
words = text.split()
inPar = False
for word in words:
    if(word[0]=='('):
        inPar = True
    if(inPar == False):
        text1 = text1+ ' ' + word
    if(word[len(word)-1]==')'):
        inPar = False    
    if(word[len(word)-2]==')'):
        inPar = False
        text1 = text1 + word[len(word)-1]
text = text1.replace('"','\'')
print(text)

 Apple is a multinational technology company that specializes in Computer services and products, which include consumer electronics, computer software, online services, software, and hardware. It designs, develops, and sells consumer electronics, computer software, and online services. The company's hardware products include the iPhone smartphone, the iPad tablet computer, the Mac personal computer, the iPod portable media player, the Apple Watch smartwatch, the digital media player, and the HomePod smart speaker. Apple's software includes the macOS and iOS operating systems, the iTunes media player, the Safari web browser, and the iLife and iWork creativity and productivity suites, as well as professional applications like Final Cut Pro, and Xcode. Its online services include the iTunes Store, the iOS App Store and Mac App Store, Apple Music, and iCloud. The company is a technological company. The company was founded by Steve Jobs. It was incorporated as Apple Computer, Inc. as the co

In [10]:
tokenized_text = word_tokenize(text)


In [11]:
classified_text = st.tag(tokenized_text)
tagged_words = []

fin_ner_text = [(l1,s1,s2) for (l1,s1),(l2,s2) in zip(classified_text,fin_ner_words)]
F_mon = ['rupees', 'rupee', 'rs', 'dollar', '$']
Org = []
Loc = []
Per = []
#print(fin_ner_text)
for word in fin_ner_text:
    lst = list(word)
    if(lst[1] == 'ORGANIZATION'):
        Org.append(lst[0].lower())
    elif(lst[1] == 'LOCATION'):
        Loc.append(lst[0].lower())
    elif(lst[1] == 'PERSON'):
        Per.append(lst[0].lower())
    elif(lst[1] == 'O' and wordnet_lemmatizer.lemmatize(lst[0]).lower() in F_mon):
        lst[1] = 'FMON'
    elif(lst[1] == 'O'):
        lst[1] = lst[2]
    word = tuple(lst)
    tagged_words.append(word)
        
for word in tagged_words:
    print((word[0],word[1]))

('Apple', 'PERSON')
('is', 'O')
('a', 'O')
('multinational', 'O')
('technology', 'O')
('company', 'O')
('that', 'O')
('specializes', 'O')
('in', 'O')
('Computer', 'PERSON')
('services', 'O')
('and', 'O')
('products', 'O')
(',', 'O')
('which', 'O')
('include', 'O')
('consumer', 'O')
('electronics', 'O')
(',', 'O')
('computer', 'O')
('software', 'O')
(',', 'O')
('online', 'O')
('services', 'O')
(',', 'O')
('software', 'O')
(',', 'O')
('and', 'O')
('hardware', 'O')
('.', 'O')
('It', 'O')
('designs', 'O')
(',', 'O')
('develops', 'O')
(',', 'O')
('and', 'O')
('sells', 'FASS')
('consumer', 'O')
('electronics', 'O')
(',', 'O')
('computer', 'O')
('software', 'O')
(',', 'O')
('and', 'O')
('online', 'O')
('services', 'O')
('.', 'O')
('The', 'O')
('company', 'O')
("'s", 'O')
('hardware', 'O')
('products', 'O')
('include', 'O')
('the', 'O')
('iPhone', 'O')
('smartphone', 'O')
(',', 'O')
('the', 'O')
('iPad', 'O')
('tablet', 'O')
('computer', 'O')
(',', 'O')
('the', 'O')
('Mac', 'PERSON')
('persona

In [12]:
quesgen = os.path.join(package_dir,'question_generation-master')
os.chdir(quesgen)

In [13]:
!./setup "$text"

data/model.t7: OK
----------------Installing python requirements----------------
You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
----------------Pulling corenlp and opennmt docker images----------------
----------------Running corenlp and opennmt servers----------------
[sudo] password for madhav: a3d613a98065b45b0b641b31d821e5640d6fd378bad517bbfbf2d7735c953e73
3c5cd96e58459686b581460072cb5fe493e5c86ee5ee6e6c533c244efddc1631
----------------Test output----------------
 Apple is a multinational technology company that specializes in Computer services and products, which include consumer electronics, computer software, online services, software, and hardware. It designs, develops, and sells consumer electronics, computer software, and online services. The company's hardware products include the iPhone smartphone, the iPad tablet computer, the Mac personal computer, the iPod portable media pl

In [14]:
from nltk import *
posmodel = os.path.join(package_dir,'stanford-postagger-full-2018-10-16/models/english-bidirectional-distsim.tagger')
posjar = os.path.join(package_dir,'stanford-postagger-full-2018-10-16/stanford-postagger.jar')
posTagger = StanfordPOSTagger(posmodel,posjar)

In [15]:
questions_path = os.path.join(package_dir,'question_generation-master/questions.txt')
lines = [line.rstrip('\n') for line in open(questions_path)]
rank_list = []

unique_list1 = []


for line in lines:
    tokenized_text = word_tokenize(line)
    capitalized_text = []
    for word in tokenized_text:
        word = word.capitalize()
        capitalized_text.append(word)
    
    classified_text = st.tag(tokenized_text)
    pos_text = posTagger.tag(tokenized_text)
    pos_ner_text = [(l1,s1,s2) for (l1,s1),(l2,s2) in zip(classified_text,pos_text)]
    
    tagged_questions = []
    count = 0
    que_end = 0
    ans = 0
    
    if(line.split('?')[0] not in unique_list1):
        unique_list1.append(line.split('?')[0])
    else:
        count = count+1
    
    for word in pos_ner_text:
        lst = list(word)
        lst_ner = list(word)
        if(lst[0] in Org):
            lst[1] = 'ORGANIZATION'
        elif(lst[0] in Loc):
            lst[1] = 'LOCATION'
        elif(lst[0] in Per):
            lst[1] = 'PERSON'
        if(lst[0] == '?'):
            que_end = 1
        elif(que_end != 1):
            if(lst[1] != 'O' or lst[2] == 'NNP'):
                count = count - 1
                #print(lst[0])
                if(lst[1] == 'ORGANIZATION'):
                    lst[0] = 'the company'
            if(lst[1] == 'O' and wordnet_lemmatizer.lemmatize(lst[0]).lower() in assets):
                lst[1] = 'FASS'
                count = count+2
            elif(lst[1] == 'O' and wordnet_lemmatizer.lemmatize(lst[0]).lower() in liabilities):
                lst[1] = 'FLIA'
                count = count+2
            elif(lst[1] == 'O' and wordnet_lemmatizer.lemmatize(lst[0]).lower() in misc1):
                lst[1] = 'FMIS'
                count = count+2
            elif(lst[1] == 'O' and wordnet_lemmatizer.lemmatize(lst[0]).lower() in F_mon):
                lst[1] = 'FMON'
                count = count+2
        elif(ans != 1):
            tag = lst[1]
            ans = 1
        word = tuple(lst)
        word_ner = tuple(lst_ner)
        tagged_questions.append(word)
    
    if(float(tokenized_text[len(classified_text)-1]) >= -5):
        tup = (line, float(tokenized_text[len(classified_text)-1]) + count, tag)
        rank_list.append(tup)
    rank_list = sorted(rank_list,key=lambda x: x[1],reverse = True)
print(rank_list)


[('what is the average profit of the company ?\t$ 800 billion\t-4.9303951263428', -0.9303951263428001, 'MONEY'), ('when was the company founded ?\tapril 1 , 1979\t-0.95197695493698', -0.95197695493698, 'DATE'), ('who was the second ceo of the company ?\tgil amelio\t-1.587094783783', -1.587094783783, 'PERSON'), ('who founded the company ?\tsteve jobs\t-3.1549923419952', -3.1549923419952, 'PERSON'), ('what was the third ceo of the company ?\tthird\t-4.7447009086609', -4.7447009086609, 'O'), ('what is the name of the company ?\tapple computer , inc.\t-4.7784662246704', -4.7784662246704, 'ORGANIZATION')]


In [16]:
unique_list = []
questions_path = os.path.join(package_dir,'question_generation-master/questions_ranked.txt')
sample = open(questions_path,'r+')
for line in rank_list:
    if(line[0].split('\t')[0] not in unique_list):
        unique_list.append(line[0].split('\t')[0])
        print("{} ~~ {}".format(line[0].split('\t')[0], line[2].split('\t')[0]),file = sample)
print(unique_list)

['what is the average profit of the company ?', 'when was the company founded ?', 'who was the second ceo of the company ?', 'who founded the company ?', 'what was the third ceo of the company ?', 'what is the name of the company ?']


In [18]:
test_path = os.path.join(package_dir,'Test.pdf')
fp1 = open(test_path, 'rb')
parser = PDFParser(fp1)
doc = PDFDocument()
parser.set_document(doc)
doc.set_parser(parser)
doc.initialize('')
rsrcmgr = PDFResourceManager()
laparams = LAParams()
laparams.char_margin = 1.0
laparams.word_margin = 1.0
device = PDFPageAggregator(rsrcmgr, laparams=laparams)
interpreter = PDFPageInterpreter(rsrcmgr, device)
extracted_text1 = ''

for page in doc.get_pages():
    interpreter.process_page(page)
    layout = device.get_result()
    for lt_obj in layout:
        if isinstance(lt_obj, LTTextBox) or isinstance(lt_obj, LTTextLine):
            extracted_text1 += lt_obj.get_text()
print(extracted_text1)

Power Finance Corporation (PFC) completed 
acquisition of majority stake in REC Ltd on 24th July. They transferred Rs 14,500 crore to the government. 
PFC is hopeful of merger of the two firms in 2019-20.  
PFC has financed 70 per cent of the deal from cash inflows and the balance 30 per cent is through debt. 
PFC borrowed around Rs 88,000 crore. 50 per cent of Rs 88,000 has been through term 
loans. The loans availed from banks are mostly based on marginal cost of funds-based lending rate (MCLR). 
PFC's loan assets would be around Rs 6 lakh crore. On the basis of 2017-18 financials, the 
consolidated annual income would be about Rs 50,000 crore and annual profit about Rs 11,000 crore. 
PFC would have higher strategic importance in financing 
of the power sector. PFC, therefore, will be a dominant player not only in the power sector but also in the entire 
financial market space. 
PFC will be the second-largest government-owned financial player in the country. Largest financial player 

In [20]:
ansext = os.path.join(package_dir,'DeepPavlov')
os.chdir(ansext)

In [21]:
from deeppavlov import build_model, configs

model = build_model(configs.squad.squad, download=False)

#context_file_name = input("Enter context file name: ")


[nltk_data] Downloading package punkt to /home/madhav/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/madhav/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/madhav/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/madhav/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
2019-05-09 22:48:02.241 INFO in 'deeppavlov.models.preprocessors.squad_preprocessor'['squad_preprocessor'] at line 310: SquadVocabEmbedder: loading saved tokens vocab from /home/madhav/.deeppavlov/models/squad_model/emb/vocab_embedder.pckl
INFO:deeppavlov.models.preprocessors.squad_preprocessor:SquadVocabEmbedder: loading saved tokens vocab from /home/madhav/.deeppavlov/models/squad_model/emb/vocab_embedder.pckl
2019-05-09 22:48:02.452 INFO

In [23]:
context = extracted_text1
questions_path = os.path.join(package_dir,'question_generation-master/questions_ranked.txt')
sample = open(questions_path,'r+')
#questions_file_name = input("Enter questions file name: ")
questions = sample.read()
questions_list = (questions.split("\n"))[0:-1]
print(questions_list)
for question in questions_list:
    x = question.split("~~")
    y = model([context], [x[0]])
    ans = y[0]
    ner_ans = st.tag(word_tokenize(ans[0]))
    if(ans in assets):
        ner_ans = "F_ASS"
    elif(ans in liabilities):
        ner_ans = "F_LIA"
    elif(ans in misc1):
        ner_ans = "F_MIS"
    elif(ans in F_mon):
        ner_ans = "F_MON"
    exp_ner = x[1]
    if(" "+ner_ans[0][1] == exp_ner):
        print(ans, ner_ans[0][1],exp_ner, 'Matched Answer Type')
    else:
        print(ans, ner_ans[0][1],exp_ner, 'Answer Type Not Matched')

['what is the average profit of the company ? ~~ MONEY', 'when was the company founded ? ~~ DATE', 'who was the second ceo of the company ? ~~ PERSON', 'who founded the company ? ~~ PERSON', 'what was the third ceo of the company ? ~~ O', 'what is the name of the company ? ~~ ORGANIZATION', '', 'what was the net revenue of the company ? ~~ MONEY', 'what was the third ceo of the company ? ~~ O', 'what is the name of the company ? ~~ ORGANIZATION', "d's largest software maker?", 'for how much did microsoft acquire linkedin?', 'how much did skype cost microsoft?']
['Rs 11,000 crore'] O  MONEY Answer Type Not Matched
['24th July'] O  DATE Answer Type Not Matched
['Power Finance Corporation'] ORGANIZATION  PERSON Answer Type Not Matched
['Power Finance Corporation'] ORGANIZATION  PERSON Answer Type Not Matched
['HDFC'] O  O Matched Answer Type


2019-05-09 22:49:49.120 INFO in 'deeppavlov.models.squad.squad'['squad'] at line 297: SQuAD model: Warning! Empty question or context was found.
INFO:deeppavlov.models.squad.squad:SQuAD model: Warning! Empty question or context was found.


['Power Finance Corporation'] ORGANIZATION  ORGANIZATION Matched Answer Type


IndexError: list index out of range